<a href="https://colab.research.google.com/github/ol3gka/OTUS_Machine-Learning.-Advanced_2023/blob/main/hw_9_Teach%20me%20senpai/RL_Taxi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Q-Learning with Taxi-v3 🚕
In this notebook, you'll implement a **Q-Learning** taxi that will need to learn to **navigate in a city to transport its passengers from point A to point B.** 🚖

## Prerequisites 🏗️
Before diving into the notebook **you need to understand**:
- The foundations of Reinforcement learning (MC, TD, Rewards hypothesis...).

[📜 Read the chapter](https://medium.com/@thomassimonini/an-introduction-to-deep-reinforcement-learning-17a565999c0c?source=friends_link&sk=1b1121ae5d9814a09ca38b47abc7dc61)

[📹 Watch the chapter](https://youtu.be/q0BiUn5LiBc)

- Q-Learning Part 1

[📜 Read the chapter](https://medium.com/@thomassimonini/q-learning-lets-create-an-autonomous-taxi-part-1-2-3e8f5e764358)

[📹 Watch the chapter](https://youtu.be/230bR2DrbdE)

- Q-Learning Part 2

[📜 Read the chapter]()




# This is a notebook from [Deep Reinforcement Learning Course v2.0 with Tensorflow and PyTorch](https://simoninithomas.github.io/deep-rl-course/)
<img src="https://simoninithomas.github.io/deep-rl-course/assets/img/environments.jpg" alt="Deep Reinforcement Course v2.0"/>
<br>

Deep RL Course v2.0 is a **free course in Deep Reinforcement Learning from beginner to expert**.

You'll build a **strong professional portfolio** by implementing awesome agents (Q-Learning, Deep Q-Learning, Policy Gradients, A2C, PPO, SAC...) **with Tensorflow and PyTorch that learns to play Space invaders👾, Doom👹, Minecraft, Starcraft, Sonic the hedgehog and more!**

In addition to the foundation's syllabus, we add a **new series on building AI for video games in Unity and Unreal Engine** using Deep RL.

## 📚 [Check the syllabus](https://simoninithomas.github.io/Deep_reinforcement_learning_Course/)
## 👨‍💻 [Check the Repository](https://github.com/simoninithomas/Deep_reinforcement_learning_Course)
## 📹 [Subscribe to our YouTube Channel](https://www.youtube.com/c/thomassimonini?sub_confirmation=1)


## Any questions ?
<p> Feel free to ask me: 📧: <a href="mailto:simonini.thomas.pro@gmail.com">simonini.thomas.pro@gmail.com</a>  </p>

## Keep in Touch  🙌

Follow me on [Twitter @ThomasSimonini](https://twitter.com/ThomasSimonini)

[Subscribe to our Youtube Channel](https://www.youtube.com/c/thomassimonini?sub_confirmation=1)



### Step 0: Install and import the libraries 📚
We will use different libraries:
- `Numpy`: For handling our Q-table.
- `OpenAI Gym`: Contains our 🚕 environment (Taxi-v3).
- `Random`: To generate random numbers (that will be useful for Epsilon-Greedy Policy).


In [ ]:
!pip install numpy
!pip install gym

https://gym.openai.com/

In [ ]:
import numpy as np
import gym
import random

## Step 1: Create the environment 🕹️
- Here we'll create the Taxi-v3 environment 🚕.

[Environment]

Our environment looks like this:
- It's a **5x5 grid world**
- Our 🚕 is spawned **randomly** in a square.
- The passenger is **spawned randomly in one of the 4 possible locations** (R, B, G, Y) and wishes to go in one of the **4 possibles locations too**.

The reward system:
- -1 for each timestep
- +20 for successfully deliver the passenger
- -10 for illegal actions (pickup or putdown the passenger at the outside of the destination).

In [ ]:
env = gym.make("Taxi-v3")
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



## Step 2: Create the Q-table and initialize it 🗄️
- Now, we'll create our Q-table, to know how much rows (states) and columns (actions) we need to know the **action_space and the observation_space**.
- OpenAI Gym provides us a way to do that: `env.action_space.n` and `env.observation_space.n`

In [ ]:
state_space = env.observation_space.n
print("There are ", state_space, " possible states")
action_space = env.action_space.n
print("There are ", action_space, " possible actions")

There are  500  possible states
There are  6  possible actions


In [ ]:
# Create our Q table with state_size rows and action_size columns (500x6)
Q = np.zeros((state_space, action_space))
print(Q)
print(Q.shape)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
(500, 6)


## Step 3: Define the hyperparameters ⚙️
- Here, we'll specify the hyperparameters

In [ ]:
total_episodes = 25000        # Total number of training episodes
total_test_episodes = 10      # Total number of test episodes
max_steps = 200               # Max steps per episode

learning_rate = 0.01          # Learning rate
gamma = 0.99                  # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.001           # Minimum exploration probability
decay_rate = 0.01             # Exponential decay rate for exploration prob

## Step 4: Define the epsilon-greedy policy 🤖

Epsilon-Greedy is a policy that handles the exploration/[exploitation trade-off](https://medium.com/@thomassimonini/an-introduction-to-deep-reinforcement-learning-17a565999c0c?source=friends_link&sk=1b1121ae5d9814a09ca38b47abc7dc61).

### The idea

Epsilon Greedy:

- *With probability 1 - ɛ* : we do **exploitation** (aka our agent selects the action with the highest state-action pair value).

- *With probability ɛ*: we do **exploration** (trying random action).

And as the training goes, **we progressively reduce the epsilon value** since we will **need less and less exploration and more exploitation**.

In [ ]:
def epsilon_greedy_policy(Q, state):
  # if random number > greater than epsilon --> exploitation
  if(random.uniform(0,1) > epsilon):
    action = np.argmax(Q[state])
  # else --> exploration
  else:
    action = env.action_space.sample()

  return action

## Step 5: Define the Q-Learning algorithm and train our agent 🧠
- Now we implement the Q learning algorithm:
[Q-Learning]

In [ ]:
 for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False

    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)

    for step in range(max_steps):
        #
        action = epsilon_greedy_policy(Q, state)

        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        Q[state][action] = Q[state][action] + learning_rate * (reward + gamma *
                                    np.max(Q[new_state]) - Q[state][action])
        # If done : finish episode
        if done == True:
            break

        # Our new state is state
        state = new_state

## Step 6: Let's watch our autonomous 🚖
- By running this cell, you'll see your smart taxi agent.




In [ ]:
import time
rewards = []

frames = []
for episode in range(total_test_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    print("****************************************************")
    print("EPISODE ", episode)
    for step in range(max_steps):
        env.render()
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(Q[state][:])
        new_state, reward, done, info = env.step(action)
        total_rewards += reward

        if done:
            rewards.append(total_rewards)
            #print ("Score", total_rewards)
            break
        state = new_state
env.close()
print ("Score over time: " +  str(sum(rewards)/total_test_episodes))

****************************************************
EPISODE  0
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
|_: | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
|_: : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+-------

In [ ]:
Q

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-3.30459614, -3.3139933 , -3.30657762, -3.31297697,  9.6218976 ,
        -3.34744808],
       [-1.46794318, -1.59695855, -1.53262033, -1.33838612, 14.11877731,
        -1.55845561],
       ...,
       [-0.78664432, -0.73092545, -0.78682865, -0.78523152, -0.78914348,
        -0.79578377],
       [-2.21453435, -2.22026663, -2.21462699, -2.22452802, -2.2819947 ,
        -2.29636785],
       [-0.0199    , -0.02989899, -0.02995701,  0.49800952, -0.199099  ,
        -0.29720701]])